In [46]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm

In [47]:
# Get the mass vs. species data
data = pd.read_csv("train_data.csv")

In [48]:
# ----------------------------Generate testing data----------------------------
test_size = 1000
# Get how many classes we need from the original data
output_dim = data.shape[0]
print("Num output classes: ", output_dim)

# Select a bunch of random rows in the data
np.random.seed(2)
masses = []
species = []
for i in range(test_size):
    rand_index = np.random.randint(output_dim)
    masses.append(data.at[rand_index, "Precise Mass"])
    # Use one-hot encodig of species strings to integers.
    species.append(rand_index)
masses = np.array(masses)

# TODO Accuracy goes down from 100% --> 0% as this uncertainty gets too small. Possibly use:
#      1) More accurate uncertainties?
#      2) A warning / default classification when vector becomes all 0s ([0,0,0,0,...0])?
# Generate noise for the same number of rows'. Multiply by an uncertainty factor representative of real data noise (e.g., usually around 10-100 ppm, or 1e-5 to 1e-4).
unc = 1e-4
noise = unc * np.random.standard_normal(size = test_size)

# Combine the noise and randomly selected masses to create our training data
x_data = masses + noise*masses
y_data = species
# print("x_data: ", x_data)
# print("y_data: ", y_data)

Num output classes:  49


In [49]:
# Make the known species masses into a row, then repeat for the length of the training data to prepare for mass array operations
# in the next cell
precise_masses = np.reshape(np.array(data["Precise Mass"]), (1,-1))
precise_masses = precise_masses.repeat(test_size,0)
print(precise_masses)

[[ 1.0073  2.0152  6.0146 ... 88.0757 91.0543 91.9719]
 [ 1.0073  2.0152  6.0146 ... 88.0757 91.0543 91.9719]
 [ 1.0073  2.0152  6.0146 ... 88.0757 91.0543 91.9719]
 ...
 [ 1.0073  2.0152  6.0146 ... 88.0757 91.0543 91.9719]
 [ 1.0073  2.0152  6.0146 ... 88.0757 91.0543 91.9719]
 [ 1.0073  2.0152  6.0146 ... 88.0757 91.0543 91.9719]]


In [50]:
# Calculate standard deviations and resulting probabilities for each mass according to its uncertainty pdf.
# We transform the x_data from 1 x n to n x 1, then broadcast it to n x 48 while subtracting precise_masses and dividing
# by the uncertainty to get the number of standard deviations each measured mass is from each of the 48 known species
# masses, then use norm.cdf to convert to probabilities.
prob_matrix = np.zeros((test_size,len(data)))
prob_matrix = (np.reshape(x_data, (-1,1)) - precise_masses) / (1000 * unc)  # TODO = 1e-1. This (overall) factor currently results in 8% accuracy @ 1e-5, 98% accuracy @ 1e-3, 97% accuracy @ 1e14, and 8% accuracy @ 1e17.
prob_matrix = (norm.cdf(-abs(prob_matrix)))

# Calculate relative probabilities by scaling row sum to 1
rel_prob_matrix = 1/np.reshape(np.sum(prob_matrix,1),(-1,1)) * prob_matrix
# Print out rows from the relative probability matrix to give us an idea of the output
print(np.round(rel_prob_matrix[0:3,:],4))

[[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  1.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.4023 0.5977 0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     1.     0.     0

In [51]:
# Now let's compare our array with the real labels and determine the classification accuracy of the Gaussian Process
y_pred = np.argmax(rel_prob_matrix, 1)
species_pred = [data.at[y, 'Species'] for y in y_pred]
species_act = [data.at[y, 'Species'] for y in y_data]

print("\nPredicted: \n", species_pred)
print("\nActual: \n", species_act)

class_acc = (y_pred == y_data).mean()*100
print("\nClassification Accuracy: ", str(class_acc) + "%")


Predicted: 
 ['Na2O+', 'C2H3+', 'C5H12N+', 'OH3+', 'K+', 'SiC3H9+', '29Si+', 'Mg+', 'Na2O+', 'NH4+', 'C4H7+', 'Na2+', 'Mg+', 'OCH3+', 'C7H7+', 'Na2+', '41K+', '30Si+', 'C3H6N+', 'CaOH+', 'Li+', 'C4H9+', 'B+', 'C4H8N+', 'SiC3H9+', 'CaOH+', 'C4H9+', 'C4H8N+', 'C4H5+', 'Li+', 'C+', 'Ca+', 'B+', 'C4H10NO+', 'CH3+', 'Na2+', 'CH4N+', 'Na2+', '6Li+', 'Si+', 'C4H10NO+', '25Mg+', 'B+', '41K+', 'C2H3+', 'CaOH+', 'C4H10NO+', 'OH3+', 'C5H12N+', 'C2H3+', 'Na2OH+', 'C5H12N+', 'OH3+', 'C2H5+', 'K+', 'Ca++', 'Na2OH+', 'C4H10NO+', '41K+', 'CH4N+', 'C2H8N+', 'SiC3H9+', 'C2H8N+', '41K+', 'OH3+', '25Mg+', 'Na+', 'Na2O+', 'C4H7+', 'Ca++', 'C3H6N+', 'CH3+', 'K+', 'CH3+', 'CH4N+', '29Si+', 'H2+', 'B+', 'Na2O+', 'C2H5+', 'CH3+', 'C3H6N+', 'C4H5+', '29Si+', '30Si+', '41K+', 'C3H3+', 'K+', 'SiC3H9+', 'C3H6N+', 'Na+', 'OH3+', '41K+', 'Fe+', 'C3H7+', 'C2H8N+', 'Si+', 'OCH3+', 'SiC3H9+', 'SiOH+', 'Si+', 'Na2O+', 'CH3+', 'Na+', 'C3H6N+', 'C2H5+', 'C2H3+', 'C2H5O+', '26Mg+', '41K+', 'CaOH+', 'SiC3H9+', 'C7H7+', '30

In [52]:
# Show some incorrect classifications and what they should be
y_pred, y_data, species_pred, species_act = np.array(y_pred), np.array(y_data), np.array(species_pred), np.array(species_act)
prediction_mask = (y_pred != y_data)

print("\nIncorrect Predictions: \n", species_pred[prediction_mask])
print("\nCorrect Species: \n", species_act[prediction_mask])

print("\nIndices of Incorrect Predictions: \n", [i for i in range(len(prediction_mask)) if prediction_mask[i]])


Incorrect Predictions: 
 ['Fe+' 'Fe+' 'Fe+']

Correct Species: 
 ['Si2+' 'Si2+' 'Si2+']

Indices of Incorrect Predictions: 
 [261, 607, 646]
